In [85]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.operatoring()

In [72]:
from gqimax.instructor import instructor_to_lut
lut = instructor_to_lut(ins)
lut.shape

(2, 4, 3, 4)

In [75]:
from gqimax.pstabilizers import PStabilizers
import cupy as cp
stb = PStabilizers(4)
stb.lambdass[0] = cp.append(stb.lambdass[0], 2)
print(stb.lambdass)
stb.indicess[0] = cp.append(stb.indicess[0], cp.expand_dims(cp.array([0, 1, 2, 3]), axis=0), axis=0)
print(stb.indicess)

[array([1., 2.]), array([1.]), array([1.]), array([1.])]
[array([[3, 0, 0, 0],
       [0, 1, 2, 3]]), array([[0, 3, 0, 0]]), array([[0, 0, 3, 0]]), array([[0, 0, 0, 3]])]


In [76]:
lambdass = stb.lambdass
indicess = stb.indicess

In [ ]:
import cupy as cp
from gqimax.mapper import weightsss_to_lambdas
from gqimax.mapper import map_cx
def map_noncx(lambdass, indicesss, LUT, k):
	# k is the index of the U operators, from outside, ranged from 0 to K-1/K'-1
	weightsss = []
	for _, indicess in enumerate(indicesss):
		# Indeciess is a k x n-dim list, example: [IXXX, YYYY] => [[0, 1, 1, 1], [2, 2, 2, 2]]
		weightss = [] # Index is a n-dim list, example: [IXXX] => [[0, 1, 1, 1]]
		for indices in indicess:
			weights = []
			for j, index in enumerate(indices): # j is the qubit index
				encoded_pauli = index - 1 # 0 is I, 1 is X, 2 is Y, 3 is Z
				if encoded_pauli == -1:
					weights.append(cp.array([1,0,0,0]))
				else:
					weights.append(LUT[k][j][encoded_pauli])
			weightss.append(weights)
		weightss = cp.array(weightss)
		weightsss.append(weightss)
	# Lambdass: n x k_i
	# Indicesss: n x k_i x n
	lambdass, indicesss = weightsss_to_lambdas(weightsss, lambdass)
	return lambdass, indicesss


# lambdass, indicesss = map_noncx(stb.indicess, stb.lambdass, lut, k = 0)

In [81]:
ins.operators[0]

[('h', 0, 0),
 ('rx', 1, 0.78),
 ('h', 2, 0),
 ('h', 0, 0),
 ('h', 2, 0),
 ('h', 2, 0),
 ('h', 3, 0),
 ('h', 3, 0),
 ('h', 3, 0),
 ('h', 3, 0),
 ('h', 3, 0)]

In [79]:
ins.xoperators

[[[0, 1], [1, 2], [1, 3]]]

In [ ]:
for j, order in enumerate(ins.orders):
	k = j // 2
	if order == 0:
		lambdass, indicesss = map_noncx(lambdass, indicess, ins.LUT, k)
	else:
		for 
		lambdass, indicesss = map_cx(lambdass, indicess, ins.xoperators[k])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [ ]:
cp.nonzero(lambdass)

(array([0, 0, 1, 1, 2, 3], dtype=int64),
 array([ 25, 192,  32,  48,   4,   1], dtype=int64))

In [ ]:
weightsss

[array([[[ 0.,  0.,  0.,  1.],
         [ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.]],
 
        [[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0., -1.,  0.],
         [ 0.,  1., -0.,  0.]]]),
 array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        , -0.70327944,  0.7109136 ],
         [ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 1.        ,  0.        ,  0.        ,  0.        ]]]),
 array([[[ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 0.,  1., -0.,  0.],
         [ 1.,  0.,  0.,  0.]]]),
 array([[[ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.],
         [ 0.,  1., -0.,  0.]]])]